Python поддерживает **наследование** (*inheritance*): класс может быть создан на основе другого класса. Это позволяет строить иерархии классов – от общего к частному.

Класс, от которого наследуют, называется **родительским** (parent class), **базовым** (base class) или **суперклассом** (*superclass*). Класс-наследник называется **дочерним** (*child class*) или **подклассом** (*subclass*).

Подкласс наследует атрибуты и методы родителя, но может их **переопределять** (*override*), задавая более специализированное поведение. Чем ниже класс в иерархии, тем более специфичным он является.

Пусть определен класс `Rectangle`:

In [17]:
class Rectangle:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def get_area(self):
        area = self.a * self.b
        return area

    def set_size(self, a=None, b=None):
        if a is not None:
            self.a = a
        if b is not None:
            self.b = b

Рассмотрим в качестве примера класс `Square`, который наследуется от класса `Rectangle`. В таком отношении наследования классов класс `Square` является подклассом, а класс `Rectangle` – суперклассом.

In [18]:
class Square(Rectangle):
    def __init__(self, a):
        Rectangle.__init__(self, a, a)

    def set_size(self, a):
        super().set_size(a, a)

    def get_perimeter(self):
        perimeter = 2 * self.a + 2 * self.b
        return perimeter

Дочерний класс `Square` получает в наследство все методы, определенные в родительском классе `Rectangle`. В классе `Square` инициализатор `__init__()` и метод `set_size()` переопределены.

### Переопределение (override)

Переопределение методов в подклассе может быть осуществлено разными способами. Если необходимо целиком изменить поведение метода, то можно его заново целиком определить. В случае, если мы хотим модифицировать поведение метода суперкласса, то вместо того, чтобы целиком его определять, лучше определить надстройки над методом и вызвать метод суперкласса, если есть такая возможность. Именно это и сделано в переопределении методов `__init__()` и `set_size()` в классе `Square`. Кроме того, было произведено **расширение** (*extension*) базового класса – добавлен метод `get_perimeter()`.

Вызвать метод суперкласса можно двумя способами:

- Напрямую через имя класса. При этом необходимо передавать первым аргументом ссылку на сам объект (`self`).

- При помощи встроенной функции `super()`, которая позволяет вызывать методы суперкласса более обобщенно. В таком способе передавать `self` в метод при вызове не нужно (он будет передан автоматически).

В нашем примере при переопределении инициализатора `__init__()` использовалось имя класса, а при переопределении метода `set_size()` использовалась функция `super()`.

> Преимуществом использования функции `super()` является то, что при изменении не придется менять имя во всех местах кода подкласса, где вызывается метод суперкласса. Однако могут возникнуть непредсказуемые проблемы при использовании множественного наследования.

### Переопределение (override)

Переопределение методов в подклассе может быть осуществлено разными способами. Если необходимо целиком изменить поведение метода, то можно определить его заново. Если же мы хотим лишь модифицировать поведение метода суперкласса, лучше вызвать метод суперкласса и добавить к нему нужную логику. Именно это и сделано в переопределении методов `__init__()` и `set_size()` в классе `Square`. Кроме того, было произведено **расширение** (*extension*) базового класса – добавлен метод `get_perimeter()`.

Вызвать метод суперкласса можно двумя способами:

- Напрямую через имя класса. При этом необходимо передавать первым аргументом ссылку на сам объект (`self`).

- При помощи встроенной функции `super()`, которая возвращает объект-посредник для вызова методов суперкласса. В этом случае передавать `self` не нужно – он будет передан автоматически.

В нашем примере при переопределении инициализатора `__init__()` использовалось имя класса, а при переопределении метода `set_size()` – функция `super()`.

> **Рекомендуется использовать `super()`** по двум причинам:
> 1. При переименовании родительского класса не придётся менять имя во всех местах кода подкласса.
> 2. При множественном наследовании `super()` корректно следует порядку разрешения методов (MRO), гарантируя, что каждый класс в иерархии будет вызван ровно один раз. Прямой вызов через имя класса может привести к повторным вызовам методов суперклассов.

Создадим экземпляр класса `Square` и вызовем его методы:

In [19]:
s1 = Square(10)
s1.set_size(20)
s1.get_area()

400

Ссылки на наследуемые классы хранятся в поле класса `__bases__` в виде кортежа:

In [20]:
Square.__bases__

(__main__.Rectangle,)

In [21]:
Square.__bases__[0] is Rectangle

True

Несмотря на то, что при определении класса `Rectangle` мы не задавали никаких классов для наследования, тем не менее, по умолчанию класс наследуется от базового класса `object`. Соответственно, в классе `Rectangle` содержатся не только поля и методы, которые мы в нем определили, но также и поля, и методы которые определены в суперклассе `object`.

In [22]:
Rectangle.__bases__

(object,)

### Порядок разрешения методов (MRO)

Иногда в случае цепочки наследований сложно отследить в каком классе определен метод, вызываемый у объекта. Для выбора метода, который будет вызван у объект, класс которого может в общем случае наследовать методы с одним и тем же именем от разных классов, Python руководствуется определенным правилом – **порядком разрешения методов** (*Method Resolution Order*).

Рассмотрим абстрактный пример наследования:

In [23]:
class A:
    def f(self): print("A.f")

class B:
    def f(self): print("B.f")

class C(A, B):
    pass

c = C()
c.f()

A.f


При вызове метода `f()` класс `C` использует метод, определенный в классе `A`, так как он шел первым аргументом при определении класса `C`.

Поле `__mro__` класса содержит последовательность наследования в виде кортежа. Метод класса `mro()` также возвращает последовательность наследования, но в виде списка. Посмотрим на порядок наследования в классе `C`:

In [24]:
C.__mro__, C.mro()

((__main__.C, __main__.A, __main__.B, object),
 [__main__.C, __main__.A, __main__.B, object])

Напишем функцию, которая для заданного объекта и имени метода возвращает класс, в котором определен метод:

In [25]:
def findDefiningClass(obj, methodName):

    typeOfObject = type(obj)            # возвращает класс объекта
    for t in typeOfObject.mro():        # перебирается список иерархии классов;
        if methodName in t.__dict__:    # если имя метода обнаруживается в
            return t                    # словаре атрибутов класса t, возвращается t

In [26]:
findDefiningClass(c, 'f')

__main__.A